# RAG 성능 평가 지표 완전 가이드

## 개요

이 노트북에서는 RAG(Retrieval-Augmented Generation) 시스템의 검색 성능을 측정하는 주요 지표들을 **이론과 실습**을 통해 완벽하게 마스터합니다.

| 주제 | 내용 |
|------|------|
| 기본 지표 | MRR, nDCG, Precision@K, Recall@K |
| 고급 지표 | F1-Score@K, MAP, Hit Rate |
| 구현 | Python으로 직접 구현 |
| 시각화 | 지표별 시각화 및 비교 |
| 실전 | 실제 RAG 시스템 평가 |

## 학습 목표

1. 각 평가 지표의 개념과 수식 이해하기
2. Python으로 평가 지표 직접 구현하기
3. 시각화를 통한 직관적 이해
4. 실제 RAG 시스템 평가에 적용하기
5. 평가 데이터셋 구축 방법 익히기

---

## 0. 필요한 라이브러리 설치 및 임포트

In [ ]:
# 필요한 패키지 설치
!pip install numpy pandas matplotlib plotly scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'AppleGothic' if plt.rcParams['font.family'] == 'sans-serif' else plt.rcParams['font.family']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 모든 라이브러리가 로드되었습니다.")

---

## 1. MRR (Mean Reciprocal Rank)

### 📚 개념

**핵심 아이디어**: "정답이 몇 번째에 나왔나?"를 점수로 바꾼 것입니다.

### 📊 점수 계산 방식

| 정답 위치 | 점수 |
|-----------|------|
| 1등 | 1/1 = **1.0** |
| 2등 | 1/2 = **0.5** |
| 3등 | 1/3 = **0.33** |
| 10등 | 1/10 = **0.1** |

정답이 위에 있을수록 점수가 높습니다.

### 🔢 공식

$$MRR = \frac{1}{Q} \times \sum_{i=1}^{Q} \frac{1}{rank_i}$$

- Q: 전체 쿼리 수
- rank_i: i번째 쿼리에서 첫 번째 정답의 순위

In [ ]:
def calculate_mrr(rankings: List[int]) -> float:
    """
    MRR (Mean Reciprocal Rank) 계산
    
    Args:
        rankings: 각 쿼리에서 첫 번째 정답의 순위 리스트
                 (정답이 없으면 0 또는 큰 수)
    
    Returns:
        MRR 점수
    """
    reciprocal_ranks = []
    
    for rank in rankings:
        if rank > 0:  # 정답이 있는 경우
            reciprocal_ranks.append(1.0 / rank)
        else:  # 정답이 없는 경우
            reciprocal_ranks.append(0.0)
    
    return np.mean(reciprocal_ranks)

# 예제: RAG 시스템에 3번 질문했을 때
example_rankings = [
    1,  # "김치 만드는 법" - 정답이 1등
    3,  # "서울 날씨" - 정답이 3등
    2,  # "파이썬 설치" - 정답이 2등
]

mrr_score = calculate_mrr(example_rankings)
print(f"예제 쿼리들의 순위: {example_rankings}")
print(f"각 쿼리의 점수: {[1/r for r in example_rankings]}")
print(f"MRR = {mrr_score:.3f}")
print(f"\n해석: {mrr_score:.3f}는 {'좋은' if mrr_score > 0.5 else '개선이 필요한'} 성능입니다.")

### MRR 시각화

In [ ]:
# MRR 점수 곡선 시각화
ranks = np.arange(1, 21)
scores = 1.0 / ranks

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ranks, y=scores,
    mode='lines+markers',
    name='MRR 점수',
    marker=dict(size=8),
    line=dict(width=2)
))

fig.update_layout(
    title='MRR 점수 곡선: 순위별 점수 변화',
    xaxis_title='정답 순위',
    yaxis_title='MRR 점수',
    hovermode='x unified',
    showlegend=True
)

# 주요 순위 표시
for i in [1, 2, 3, 5, 10]:
    fig.add_annotation(
        x=i, y=1/i,
        text=f"{1/i:.2f}",
        showarrow=False,
        yshift=10
    )

fig.show()

print("💡 인사이트:")
print("- 1등과 2등의 차이(0.5)가 2등과 3등의 차이(0.17)보다 훨씬 큽니다.")
print("- 상위 랭킹이 매우 중요합니다!")

---

## 2. nDCG (Normalized Discounted Cumulative Gain)

### 📚 개념

**핵심 아이디어**: "관련성 높은 결과가 위에 있을수록 좋다"를 점수로 바꾼 것입니다.

### 🆚 MRR과의 차이점

| | MRR | nDCG |
|---|-----|------|
| 정답 개수 | 1개만 봄 | **여러 개** 봄 |
| 관련성 | 정답/오답 (0 or 1) | **등급** (0, 1, 2, 3...) |

### 🔢 계산 과정 (3단계)

In [ ]:
def calculate_dcg(relevances: List[float], k: int = None) -> float:
    """
    DCG (Discounted Cumulative Gain) 계산
    
    Args:
        relevances: 각 문서의 관련성 점수 리스트
        k: 상위 k개만 고려 (None이면 전체)
    
    Returns:
        DCG 점수
    """
    if k is None:
        k = len(relevances)
    
    dcg = 0.0
    for i in range(min(k, len(relevances))):
        # 위치 i+1의 할인율 적용 (i는 0부터 시작)
        dcg += relevances[i] / np.log2(i + 2)  # i+2는 position이 1부터 시작하기 때문
    
    return dcg

def calculate_ndcg(relevances: List[float], k: int = None) -> float:
    """
    nDCG (Normalized DCG) 계산
    
    Args:
        relevances: 각 문서의 관련성 점수 리스트
        k: 상위 k개만 고려
    
    Returns:
        nDCG 점수 (0~1)
    """
    # 실제 DCG 계산
    dcg = calculate_dcg(relevances, k)
    
    # 이상적인 순서로 정렬하여 IDCG 계산
    ideal_relevances = sorted(relevances, reverse=True)
    idcg = calculate_dcg(ideal_relevances, k)
    
    # IDCG가 0이면 nDCG도 0
    if idcg == 0:
        return 0.0
    
    return dcg / idcg

# 예제: 검색 결과 5개의 관련성 점수
relevances = [3, 1, 2, 0, 1]  # 실제 순서
ideal_order = sorted(relevances, reverse=True)  # [3, 2, 1, 1, 0]

dcg = calculate_dcg(relevances)
idcg = calculate_dcg(ideal_order)
ndcg = calculate_ndcg(relevances)

print("📊 nDCG 계산 과정:")
print(f"\n1. 실제 관련성 순서: {relevances}")
print(f"2. 이상적인 순서: {ideal_order}")
print(f"\n3. DCG = {dcg:.3f}")
print(f"4. IDCG = {idcg:.3f}")
print(f"5. nDCG = DCG/IDCG = {ndcg:.3f}")
print(f"\n해석: {ndcg:.3f}는 {'좋은' if ndcg > 0.8 else '개선이 필요한'} 순서입니다.")

### nDCG 시각화

In [ ]:
# 여러 검색 결과의 nDCG 비교
scenarios = {
    '완벽한 순서': [3, 2, 1, 1, 0],
    '좋은 순서': [3, 1, 2, 0, 1],
    '나쁜 순서': [0, 1, 1, 2, 3],
    '랜덤 순서': [1, 0, 3, 1, 2]
}

results = []
for name, rels in scenarios.items():
    ndcg = calculate_ndcg(rels)
    results.append({'시나리오': name, 'nDCG': ndcg, '관련성 순서': str(rels)})

df_ndcg = pd.DataFrame(results)

# 막대 그래프
fig = px.bar(df_ndcg, x='시나리오', y='nDCG', 
             hover_data=['관련성 순서'],
             title='시나리오별 nDCG 점수 비교',
             color='nDCG',
             color_continuous_scale='RdYlGn')

fig.update_layout(yaxis_range=[0, 1.1])
fig.add_hline(y=0.8, line_dash="dash", line_color="green", 
              annotation_text="좋은 성능 (0.8)")
fig.add_hline(y=0.5, line_dash="dash", line_color="orange", 
              annotation_text="개선 필요 (0.5)")

fig.show()

print("\n📊 결과 테이블:")
print(df_ndcg.to_string(index=False))

---

## 3. Precision@K와 Recall@K

### 📚 개념

- **Precision@K**: "가져온 K개 중에 정답이 몇 개?"
- **Recall@K**: "전체 정답 중에 K개 안에 몇 개 들어왔나?"

### 🔢 공식

$$Precision@K = \frac{\text{상위 K개 중 정답 수}}{K}$$

$$Recall@K = \frac{\text{상위 K개 중 정답 수}}{\text{전체 정답 수}}$$

In [ ]:
def calculate_precision_at_k(retrieved: List[int], relevant: List[int], k: int) -> float:
    """
    Precision@K 계산
    
    Args:
        retrieved: 검색된 문서 ID 리스트 (순서대로)
        relevant: 관련 문서 ID 리스트
        k: 상위 k개 고려
    
    Returns:
        Precision@K 점수
    """
    retrieved_k = retrieved[:k]
    relevant_set = set(relevant)
    
    # 상위 k개 중 관련 문서 개수
    relevant_in_k = sum(1 for doc in retrieved_k if doc in relevant_set)
    
    return relevant_in_k / k if k > 0 else 0

def calculate_recall_at_k(retrieved: List[int], relevant: List[int], k: int) -> float:
    """
    Recall@K 계산
    
    Args:
        retrieved: 검색된 문서 ID 리스트 (순서대로)
        relevant: 관련 문서 ID 리스트
        k: 상위 k개 고려
    
    Returns:
        Recall@K 점수
    """
    retrieved_k = retrieved[:k]
    relevant_set = set(relevant)
    
    # 상위 k개 중 관련 문서 개수
    relevant_in_k = sum(1 for doc in retrieved_k if doc in relevant_set)
    
    return relevant_in_k / len(relevant_set) if len(relevant_set) > 0 else 0

# 예제: RAG 검색 결과
retrieved_docs = [1, 3, 5, 7, 9, 2, 4, 6, 8, 10]  # 검색된 문서 ID (순서대로)
relevant_docs = [1, 2, 5, 8, 11, 13, 15, 17]  # 실제 관련 문서 ID

# K=5일 때 계산
k = 5
precision_5 = calculate_precision_at_k(retrieved_docs, relevant_docs, k)
recall_5 = calculate_recall_at_k(retrieved_docs, relevant_docs, k)

print(f"📊 검색 결과 분석 (K={k}):")
print(f"\n검색된 상위 {k}개: {retrieved_docs[:k]}")
print(f"전체 관련 문서: {relevant_docs}")
print(f"상위 {k}개 중 정답: {[doc for doc in retrieved_docs[:k] if doc in relevant_docs]}")
print(f"\nPrecision@{k} = {precision_5:.2f} ({precision_5*100:.0f}%)")
print(f"  → 가져온 {k}개 중 {precision_5*k:.0f}개가 정답")
print(f"\nRecall@{k} = {recall_5:.2f} ({recall_5*100:.0f}%)")
print(f"  → 전체 정답 {len(relevant_docs)}개 중 {recall_5*len(relevant_docs):.0f}개를 찾음")

### Precision-Recall 트레이드오프 시각화

In [ ]:
# K값 변화에 따른 Precision과 Recall 변화
k_values = range(1, 11)
precisions = []
recalls = []

for k in k_values:
    p = calculate_precision_at_k(retrieved_docs, relevant_docs, k)
    r = calculate_recall_at_k(retrieved_docs, relevant_docs, k)
    precisions.append(p)
    recalls.append(r)

# 그래프 생성
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(k_values), y=precisions,
    mode='lines+markers',
    name='Precision@K',
    marker=dict(size=8)
))

fig.add_trace(go.Scatter(
    x=list(k_values), y=recalls,
    mode='lines+markers',
    name='Recall@K',
    marker=dict(size=8)
))

fig.update_layout(
    title='Precision@K vs Recall@K: 트레이드오프',
    xaxis_title='K (검색 결과 개수)',
    yaxis_title='점수',
    hovermode='x unified',
    yaxis_range=[0, 1]
)

fig.show()

print("💡 인사이트:")
print("- K가 증가하면 Recall은 올라가지만 Precision은 떨어집니다.")
print("- 적절한 K값 선택이 중요합니다!")

---

## 4. F1-Score@K

### 📚 개념

Precision과 Recall의 조화평균으로, 두 지표의 균형을 측정합니다.

### 🔢 공식

$$F1@K = 2 \times \frac{Precision@K \times Recall@K}{Precision@K + Recall@K}$$

In [ ]:
def calculate_f1_at_k(retrieved: List[int], relevant: List[int], k: int) -> float:
    """
    F1-Score@K 계산
    
    Args:
        retrieved: 검색된 문서 ID 리스트
        relevant: 관련 문서 ID 리스트
        k: 상위 k개 고려
    
    Returns:
        F1@K 점수
    """
    precision = calculate_precision_at_k(retrieved, relevant, k)
    recall = calculate_recall_at_k(retrieved, relevant, k)
    
    if precision + recall == 0:
        return 0
    
    return 2 * (precision * recall) / (precision + recall)

# K값별 F1-Score 계산
f1_scores = []
for k in k_values:
    f1 = calculate_f1_at_k(retrieved_docs, relevant_docs, k)
    f1_scores.append(f1)

# 모든 지표 함께 시각화
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(k_values), y=precisions, mode='lines+markers', name='Precision@K'))
fig.add_trace(go.Scatter(x=list(k_values), y=recalls, mode='lines+markers', name='Recall@K'))
fig.add_trace(go.Scatter(x=list(k_values), y=f1_scores, mode='lines+markers', name='F1-Score@K',
                        line=dict(width=3)))

fig.update_layout(
    title='검색 성능 지표 종합 비교',
    xaxis_title='K (검색 결과 개수)',
    yaxis_title='점수',
    hovermode='x unified',
    yaxis_range=[0, 1]
)

fig.show()

# 최적 K 찾기
best_k = k_values[np.argmax(f1_scores)]
best_f1 = max(f1_scores)

print(f"\n🎯 최적 K값: {best_k} (F1-Score: {best_f1:.3f})")
print(f"   - Precision@{best_k}: {precisions[best_k-1]:.3f}")
print(f"   - Recall@{best_k}: {recalls[best_k-1]:.3f}")

---

## 5. MAP (Mean Average Precision)

### 📚 개념

Average Precision의 평균으로, 순서를 고려한 Precision의 종합 지표입니다.

### 🔢 공식

$$AP = \frac{1}{|R|} \sum_{k=1}^{n} P(k) \times rel(k)$$

- P(k): k번째까지의 Precision
- rel(k): k번째 문서가 관련있으면 1, 아니면 0
- |R|: 전체 관련 문서 수

In [ ]:
def calculate_average_precision(retrieved: List[int], relevant: List[int]) -> float:
    """
    Average Precision 계산
    
    Args:
        retrieved: 검색된 문서 ID 리스트
        relevant: 관련 문서 ID 리스트
    
    Returns:
        Average Precision 점수
    """
    relevant_set = set(relevant)
    if not relevant_set:
        return 0.0
    
    ap_sum = 0.0
    relevant_found = 0
    
    for i, doc_id in enumerate(retrieved, 1):
        if doc_id in relevant_set:
            relevant_found += 1
            precision_at_i = relevant_found / i
            ap_sum += precision_at_i
    
    return ap_sum / len(relevant_set)

def calculate_map(queries_results: List[Tuple[List[int], List[int]]]) -> float:
    """
    MAP (Mean Average Precision) 계산
    
    Args:
        queries_results: [(retrieved, relevant)] 튜플의 리스트
    
    Returns:
        MAP 점수
    """
    aps = []
    for retrieved, relevant in queries_results:
        ap = calculate_average_precision(retrieved, relevant)
        aps.append(ap)
    
    return np.mean(aps)

# 예제: 3개 쿼리의 결과
queries = [
    # (검색 결과, 정답 문서)
    ([1, 3, 5, 2, 4], [1, 2, 5]),        # 쿼리 1
    ([10, 11, 12, 13, 14], [11, 13, 15]), # 쿼리 2
    ([20, 21, 22, 23, 24], [20, 21, 22])  # 쿼리 3
]

# 각 쿼리의 AP 계산
print("📊 각 쿼리의 Average Precision:\n")
aps = []
for i, (retrieved, relevant) in enumerate(queries, 1):
    ap = calculate_average_precision(retrieved, relevant)
    aps.append(ap)
    print(f"쿼리 {i}:")
    print(f"  검색 결과: {retrieved}")
    print(f"  정답 문서: {relevant}")
    print(f"  AP = {ap:.3f}\n")

# MAP 계산
map_score = calculate_map(queries)
print(f"\n🎯 MAP (Mean Average Precision) = {map_score:.3f}")
print(f"\n해석: MAP {map_score:.3f}는 {'우수한' if map_score > 0.7 else '개선이 필요한'} 성능입니다.")

---

## 6. Hit Rate (Success Rate)

### 📚 개념

상위 K개 중에 최소 1개 이상의 정답이 있는 쿼리의 비율입니다.
가장 단순하지만 직관적인 지표입니다.

In [ ]:
def calculate_hit_rate(queries_results: List[Tuple[List[int], List[int]]], k: int) -> float:
    """
    Hit Rate@K 계산
    
    Args:
        queries_results: [(retrieved, relevant)] 튜플의 리스트
        k: 상위 k개 고려
    
    Returns:
        Hit Rate 점수
    """
    hits = 0
    
    for retrieved, relevant in queries_results:
        retrieved_k = set(retrieved[:k])
        relevant_set = set(relevant)
        
        if retrieved_k & relevant_set:  # 교집합이 있으면 hit
            hits += 1
    
    return hits / len(queries_results)

# Hit Rate 계산
k_values = [1, 3, 5, 10]
hit_rates = []

print("📊 Hit Rate@K 분석:\n")
for k in k_values:
    hit_rate = calculate_hit_rate(queries, k)
    hit_rates.append(hit_rate)
    print(f"Hit Rate@{k}: {hit_rate:.2f} ({hit_rate*100:.0f}%)")

# 시각화
fig = go.Figure(data=[go.Bar(x=[f"@{k}" for k in k_values], y=hit_rates)])
fig.update_layout(
    title='Hit Rate@K: K값에 따른 성공률',
    xaxis_title='K',
    yaxis_title='Hit Rate',
    yaxis_range=[0, 1.1]
)
fig.show()

---

## 7. 실제 RAG 시스템 평가 예제

실제 RAG 시스템을 평가하는 전체 파이프라인을 구현해봅시다.

In [ ]:
class RAGEvaluator:
    """
    RAG 시스템 평가 클래스
    """
    
    def __init__(self, name: str = "RAG System"):
        self.name = name
        self.results = []
    
    def add_query_result(self, query: str, retrieved: List[int], relevant: List[int]):
        """
        쿼리 결과 추가
        """
        self.results.append({
            'query': query,
            'retrieved': retrieved,
            'relevant': relevant
        })
    
    def evaluate(self, k: int = 5) -> Dict[str, float]:
        """
        모든 지표 계산
        """
        if not self.results:
            return {}
        
        metrics = {
            'MRR': [],
            'nDCG': [],
            'Precision': [],
            'Recall': [],
            'F1': [],
            'AP': []
        }
        
        for result in self.results:
            retrieved = result['retrieved']
            relevant = result['relevant']
            
            # MRR 계산
            first_relevant = None
            for i, doc in enumerate(retrieved, 1):
                if doc in relevant:
                    first_relevant = i
                    break
            mrr = 1/first_relevant if first_relevant else 0
            metrics['MRR'].append(mrr)
            
            # 다른 지표들
            metrics['Precision'].append(calculate_precision_at_k(retrieved, relevant, k))
            metrics['Recall'].append(calculate_recall_at_k(retrieved, relevant, k))
            metrics['F1'].append(calculate_f1_at_k(retrieved, relevant, k))
            metrics['AP'].append(calculate_average_precision(retrieved, relevant))
            
            # nDCG (간단히 binary relevance로)
            relevances = [1 if doc in relevant else 0 for doc in retrieved[:k]]
            metrics['nDCG'].append(calculate_ndcg(relevances, k))
        
        # 평균 계산
        avg_metrics = {}
        for metric_name, values in metrics.items():
            avg_metrics[metric_name] = np.mean(values)
        
        # Hit Rate
        queries_data = [(r['retrieved'], r['relevant']) for r in self.results]
        avg_metrics['Hit_Rate'] = calculate_hit_rate(queries_data, k)
        
        return avg_metrics
    
    def print_report(self, k: int = 5):
        """
        평가 리포트 출력
        """
        metrics = self.evaluate(k)
        
        print(f"\n{'='*60}")
        print(f"📊 {self.name} 평가 리포트 (K={k})")
        print(f"{'='*60}\n")
        
        for metric, value in metrics.items():
            bar = '█' * int(value * 20)
            print(f"{metric:12} : {value:.3f}  {bar}")
        
        print(f"\n총 {len(self.results)}개 쿼리 평가 완료")

# 실제 RAG 시스템 시뮬레이션
evaluator = RAGEvaluator("My RAG System")

# 샘플 쿼리 결과들 (실제로는 RAG 시스템에서 가져옴)
sample_queries = [
    {
        'query': "파이썬에서 리스트 정렬하는 방법?",
        'retrieved': [1, 5, 3, 7, 9, 2, 4, 6, 8, 10],
        'relevant': [1, 3, 5, 8]
    },
    {
        'query': "머신러닝 모델 평가 지표는?",
        'retrieved': [12, 14, 11, 15, 13, 16, 17, 18, 19, 20],
        'relevant': [11, 12, 15, 17, 21]
    },
    {
        'query': "딥러닝 최적화 알고리즘 종류?",
        'retrieved': [22, 25, 23, 24, 21, 26, 27, 28, 29, 30],
        'relevant': [22, 23, 25, 26, 31, 32]
    }
]

# 결과 추가
for q in sample_queries:
    evaluator.add_query_result(q['query'], q['retrieved'], q['relevant'])

# 평가 리포트 출력
evaluator.print_report(k=5)

---

## 8. 평가 지표 종합 비교

모든 지표를 한눈에 비교해봅시다.

In [ ]:
# 두 가지 RAG 시스템 비교
system_a = RAGEvaluator("System A (Baseline)")
system_b = RAGEvaluator("System B (Improved)")

# System A 결과 (기본 시스템)
for q in sample_queries:
    # 랜덤하게 섞인 결과 시뮬레이션
    retrieved = list(range(1, 31))
    np.random.shuffle(retrieved)
    system_a.add_query_result(q['query'], retrieved[:10], q['relevant'])

# System B 결과 (개선된 시스템)
for q in sample_queries:
    # 관련 문서를 더 앞쪽에 배치
    relevant_docs = q['relevant']
    other_docs = [i for i in range(1, 31) if i not in relevant_docs]
    retrieved = relevant_docs[:3] + other_docs[:7]
    np.random.shuffle(retrieved[:5])  # 상위 5개만 약간 섞기
    system_b.add_query_result(q['query'], retrieved, q['relevant'])

# 비교 그래프
metrics_a = system_a.evaluate(k=5)
metrics_b = system_b.evaluate(k=5)

metric_names = list(metrics_a.keys())
values_a = list(metrics_a.values())
values_b = list(metrics_b.values())

fig = go.Figure()
fig.add_trace(go.Bar(name='System A (Baseline)', x=metric_names, y=values_a))
fig.add_trace(go.Bar(name='System B (Improved)', x=metric_names, y=values_b))

fig.update_layout(
    title='RAG 시스템 성능 비교',
    yaxis_title='점수',
    barmode='group',
    yaxis_range=[0, 1]
)

fig.show()

# 개선율 계산
print("\n📈 개선율 분석:")
for metric in metric_names:
    improvement = (metrics_b[metric] - metrics_a[metric]) / metrics_a[metric] * 100 if metrics_a[metric] > 0 else 0
    arrow = "↑" if improvement > 0 else "↓" if improvement < 0 else "="
    print(f"{metric:12}: {improvement:+.1f}% {arrow}")

---

## 9. 평가 데이터셋 구축 방법

실제 RAG 시스템을 평가하기 위한 데이터셋 구축 방법을 알아봅시다.

In [ ]:
class RAGTestDataset:
    """
    RAG 평가용 데이터셋 클래스
    """
    
    def __init__(self):
        self.queries = []
        self.documents = {}
        self.relevance_judgments = {}
    
    def add_document(self, doc_id: int, content: str, metadata: dict = None):
        """
        문서 추가
        """
        self.documents[doc_id] = {
            'content': content,
            'metadata': metadata or {}
        }
    
    def add_query(self, query_id: int, query_text: str, relevant_docs: List[int], 
                  relevance_scores: Dict[int, float] = None):
        """
        쿼리와 관련 문서 추가
        
        Args:
            query_id: 쿼리 ID
            query_text: 쿼리 텍스트
            relevant_docs: 관련 문서 ID 리스트
            relevance_scores: 문서별 관련성 점수 (선택적)
        """
        self.queries.append({
            'id': query_id,
            'text': query_text
        })
        
        self.relevance_judgments[query_id] = {
            'relevant_docs': relevant_docs,
            'scores': relevance_scores or {doc_id: 1.0 for doc_id in relevant_docs}
        }
    
    def get_statistics(self) -> dict:
        """
        데이터셋 통계
        """
        total_relevant = sum(len(j['relevant_docs']) 
                           for j in self.relevance_judgments.values())
        
        return {
            'num_queries': len(self.queries),
            'num_documents': len(self.documents),
            'avg_relevant_per_query': total_relevant / len(self.queries) if self.queries else 0,
            'total_relevance_judgments': total_relevant
        }
    
    def export_to_json(self, filepath: str):
        """
        JSON으로 내보내기
        """
        import json
        
        data = {
            'queries': self.queries,
            'documents': self.documents,
            'relevance_judgments': self.relevance_judgments
        }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

# 평가 데이터셋 구축 예제
dataset = RAGTestDataset()

# 문서 추가
documents = [
    (1, "Python은 간단하고 읽기 쉬운 프로그래밍 언어입니다."),
    (2, "리스트는 Python의 기본 자료구조 중 하나입니다."),
    (3, "정렬 알고리즘에는 퀵소트, 머지소트, 힙소트 등이 있습니다."),
    (4, "머신러닝은 데이터로부터 패턴을 학습하는 기술입니다."),
    (5, "딥러닝은 신경망을 여러 층으로 쌓은 머신러닝 기법입니다."),
]

for doc_id, content in documents:
    dataset.add_document(doc_id, content)

# 쿼리와 관련성 판단 추가
dataset.add_query(
    query_id=1,
    query_text="Python 리스트 정렬 방법",
    relevant_docs=[2, 3],
    relevance_scores={2: 1.0, 3: 0.8}
)

dataset.add_query(
    query_id=2,
    query_text="머신러닝과 딥러닝의 차이",
    relevant_docs=[4, 5],
    relevance_scores={4: 1.0, 5: 1.0}
)

# 통계 출력
stats = dataset.get_statistics()
print("📊 평가 데이터셋 통계:\n")
for key, value in stats.items():
    print(f"  {key}: {value}")

print("\n💡 평가 데이터셋 구축 팁:")
print("1. 다양한 난이도의 쿼리 포함")
print("2. 실제 사용자 질문 기반")
print("3. 전문가의 관련성 판단")
print("4. 정기적인 업데이트")
print("5. 편향 제거")

---

## 10. 지표 선택 가이드

### 상황별 추천 지표

In [ ]:
# 지표 선택 가이드 테이블
guide_data = [
    {
        '상황': '단일 정답을 빠르게 찾기',
        '추천 지표': 'MRR',
        '이유': '첫 번째 정답의 위치가 중요',
        '예시': 'FAQ 검색, 정의 찾기'
    },
    {
        '상황': '여러 관련 문서의 순서 중요',
        '추천 지표': 'nDCG',
        '이유': '관련성 등급과 순서를 함께 고려',
        '예시': '제품 추천, 뉴스 검색'
    },
    {
        '상황': '정확한 결과만 필요',
        '추천 지표': 'Precision@K',
        '이유': '노이즈를 최소화해야 함',
        '예시': '의료 정보, 법률 문서'
    },
    {
        '상황': '모든 관련 문서를 찾아야 함',
        '추천 지표': 'Recall@K',
        '이유': '놓치는 정보가 없어야 함',
        '예시': '특허 검색, 연구 논문'
    },
    {
        '상황': '균형잡힌 성능 필요',
        '추천 지표': 'F1-Score@K',
        '이유': 'Precision과 Recall의 균형',
        '예시': '일반적인 RAG 시스템'
    },
    {
        '상황': '전체 랭킹 품질 평가',
        '추천 지표': 'MAP',
        '이유': '순서를 고려한 종합 평가',
        '예시': '검색 엔진 평가'
    },
]

df_guide = pd.DataFrame(guide_data)

# 테이블 시각화
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>상황</b>', '<b>추천 지표</b>', '<b>이유</b>', '<b>예시</b>'],
        fill_color='#4CAF50',
        align='left',
        font=dict(color='white', size=12)
    ),
    cells=dict(
        values=[df_guide[col] for col in df_guide.columns],
        fill_color=['#f1f1f1', '#e8f5e9', '#f1f1f1', '#e8f5e9'],
        align='left',
        height=30
    )
)])

fig.update_layout(
    title='RAG 평가 지표 선택 가이드',
    height=400,
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.show()

---

## 11. 실전 팁과 베스트 프랙티스

### 🎯 RAG 평가 체크리스트

In [ ]:
print("""
✅ RAG 평가 체크리스트
========================

📊 평가 준비
□ 대표성 있는 쿼리 세트 준비 (최소 50개)
□ 전문가의 관련성 판단 확보
□ 다양한 난이도와 유형의 쿼리 포함
□ 실제 사용 환경과 유사한 조건 설정

📈 지표 선택
□ 비즈니스 목표에 맞는 주요 지표 선정
□ 보조 지표 2-3개 추가 선정
□ 사용자 만족도와의 상관관계 검증

🔬 실험 설계
□ A/B 테스트 설정
□ 통계적 유의성 검증 계획
□ 온라인/오프라인 평가 병행

📝 결과 분석
□ 실패 케이스 분석 (Error Analysis)
□ 개선 포인트 도출
□ 사용자 피드백 수집

🚀 최적화
□ 청킹 전략 조정
□ 임베딩 모델 비교
□ 검색 알고리즘 개선
□ 하이브리드 검색 적용
""")

print("\n" + "="*60)
print("\n💡 추가 개선 아이디어:\n")

improvements = [
    "1. **하이브리드 검색**: BM25 + Vector Search 조합",
    "2. **재순위화**: Cross-Encoder로 상위 결과 재정렬",
    "3. **쿼리 확장**: 동의어/관련어 추가",
    "4. **적응형 K**: 쿼리 복잡도에 따라 K값 동적 조정",
    "5. **캐싱**: 자주 검색되는 쿼리 결과 캐싱",
    "6. **피드백 루프**: 사용자 클릭 데이터로 지속 개선"
]

for improvement in improvements:
    print(f"  {improvement}")

---

## 12. 요약 및 결론

### 📌 핵심 정리

In [ ]:
# 최종 요약 테이블
summary_data = {
    '지표': ['MRR', 'nDCG', 'Precision@K', 'Recall@K', 'F1@K', 'MAP', 'Hit Rate'],
    '핵심 질문': [
        '첫 정답이 몇 번째?',
        '관련성 높은 게 위에?',
        'K개 중 정답 비율?',
        '전체 정답 중 찾은 비율?',
        'P와 R의 균형은?',
        '전체 순위 품질은?',
        '최소 1개는 찾았나?'
    ],
    '범위': ['0~1', '0~1', '0~1', '0~1', '0~1', '0~1', '0~1'],
    '복잡도': ['낮음', '높음', '낮음', '낮음', '낮음', '중간', '낮음'],
    '우선 사용': [
        '단일 정답',
        '랭킹 품질',
        '정확도 중시',
        '완전성 중시',
        '균형',
        '종합 평가',
        '기본 체크'
    ]
}

df_summary = pd.DataFrame(summary_data)

print("📊 RAG 평가 지표 총정리\n")
print(df_summary.to_string(index=False))

print("\n" + "="*60)
print("""
🎯 핵심 메시지:

1. **하나의 지표로는 부족합니다**
   - 여러 지표를 종합적으로 봐야 합니다
   
2. **비즈니스 목표가 우선입니다**
   - 사용자 만족도와 연결되는 지표를 선택하세요
   
3. **지속적인 평가가 중요합니다**
   - 정기적으로 평가하고 개선하세요
   
4. **실제 데이터로 평가하세요**
   - 합성 데이터보다 실제 사용자 쿼리가 중요합니다

모든 지표는 1에 가까울수록 좋은 성능을 의미합니다! 🚀
""")

---

## 참고 자료

- [Information Retrieval 교과서](https://nlp.stanford.edu/IR-book/)
- [TREC (Text REtrieval Conference)](https://trec.nist.gov/)
- [MS MARCO 데이터셋](https://microsoft.github.io/msmarco/)
- [BEIR Benchmark](https://github.com/beir-cellar/beir)
- [LangChain Evaluation](https://python.langchain.com/docs/guides/evaluation)
- [Ragas: RAG 평가 프레임워크](https://github.com/explodinggradients/ragas)

---

**축하합니다! 🎉**

RAG 시스템 평가 지표를 완벽하게 마스터했습니다.
이제 여러분의 RAG 시스템을 정확하게 평가하고 개선할 수 있습니다!